## Output Parser解析器

1. 引入库

In [14]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import XMLOutputParser
from pprint import pprint
from chat_model_client import get_model

2. 声明返回结构

In [15]:
class Marathon(BaseModel):
    name: str = Field(
        default="",
        description="name of event"
    ),
    date: str = Field(
        default="",
        description="the date of event"
    ),
    city: str = Field(
        default="",
        description="Event host city"
    )

3.解析器及prompt

In [16]:
query = "随机生成一个马拉松赛事，并给出比赛时间及城市"
parser = XMLOutputParser(pydantic_object=Marathon)
prompt = PromptTemplate(
    template="回答下面的问题：\n{format_instructions}\n{query}\n如果输出的是代码块，请不要包括首尾的```符号",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

4. Prompt

In [18]:
pprint(prompt.format(query=query))

('回答下面的问题：\n'
 'The output should be formatted as a XML file.\n'
 '1. Output should conform to the tags below.\n'
 '2. If tags are not given, make them on your own.\n'
 '3. Remember to always open and close all the tags.\n'
 '\n'
 'As an example, for the tags ["foo", "bar", "baz"]:\n'
 '1. String "<foo>\n'
 '   <bar>\n'
 '      <baz></baz>\n'
 '   </bar>\n'
 '</foo>" is a well-formatted instance of the schema.\n'
 '2. String "<foo>\n'
 '   <bar>\n'
 '   </foo>" is a badly-formatted instance.\n'
 '3. String "<foo>\n'
 '   <tag>\n'
 '   </tag>\n'
 '</foo>" is a badly-formatted instance.\n'
 '\n'
 'Here are the output tags:\n'
 '```\n'
 'None\n'
 '```\n'
 '随机生成一个马拉松赛事，并给出比赛时间及城市\n'
 '如果输出的是代码块，请不要包括首尾的```符号')


5. 解析并输出内容

In [17]:
llm = get_model("openai")
chain = prompt | llm | parser
chain.invoke(query)

/var/folders/5x/c0q41fpx6l540lsl42_bzk5h0000gq/T/ipykernel_61871/4044007954.py:1: UserWarning: Parameters {'presence_penalty', 'frequency_penalty', 'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  llm = get_model("openai")


{'marathon': [{'event': 'City Marathon'},
  {'date': 'October 15, 2022'},
  {'city': 'New York City'}]}